In [2]:
import os, sys
import numpy as np
import json
import random
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import torchvision
from torchvision import datasets, models, transforms
from numpy import linalg as LA
sys.path.append('../environments/')
from sepsis_env import SepsisEnv

In [3]:
env = SepsisEnv()
fg_tuples = env.generate_tuples(group="foreground", n_trajectories=2)
bg_tuples = env.generate_tuples(group="background", n_trajectories=2)

In [4]:
all_tuples = bg_tuples + fg_tuples
random.shuffle(all_tuples)
split = 0.8
train_tuples = all_tuples[:int(split*len(all_tuples))]
test_tuples = all_tuples[int(split*len(all_tuples)):]

In [19]:
one_hot_a = [0]*25
s = train_tuples[0][0]
len(s)
a = train_tuples[0][1]
one_hot_a[a] = 1

46

In [21]:
class ContrastiveDataset(Dataset):
    def __init__(self, tuples):
        self.tuples = tuples
            
        def construct_pairs(tuples):
            X = []
            y = []
            for t in self.tuples:
                if t[4] == 'background':
                    one_hot_a = [0]*25
                    s = t[0]
                    a = t[1]
                    one_hot_a[a] = 1
                    r = t[3][0]
                    
                    blank_s = [0]*46
                    blank_a = [0]
                    s_a = np.hstack((s, a, blank_s, blank_a))
                    X.append(s_a)
                    y.append(r)
                else:
                    one_hot_a = [0]*25
                    s = t[0]
                    a = t[1]
                    one_hot_a[a] = 1
                    r = t[3][0]
                    
                    s_a = np.hstack((s, a, s, a))
                    X.append(s, a)
                    y.append(r)
        return X, y
    
        self.X, self.y = self.construct_pairs(self.tuples)

    def __len__(self):
        return len(self.tuples)

    def __getitem__(self,idx):
        return (self.X[idx], self.y[idx])